# Домашнее задание № 2. Мешок слов. (Шеин Александр)

## Задание 1

In [22]:
import pandas as pd
from razdel import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from string import punctuation


def use_vectorizer(vectorizer, train, test):
    X_train = vectorizer.fit_transform(train.comment)
    X_test = vectorizer.transform(test.comment)
    y_train = train.toxic.values
    y_test = test.toxic.values
    classifier = MultinomialNB(fit_prior=False)
    classifier.fit(X_train, y_train)
    predictions = classifier.predict(X_test)
    print(classification_report(y_test, predictions))


def run_experiment():
    data = pd.read_csv('labeled.csv')
    train, test = train_test_split(data, test_size=0.1)
    train.reset_index(inplace=True)
    test.reset_index(inplace=True)
    default_vectorizer = TfidfVectorizer(min_df=5, max_df=0.6)
    custom_vectorizer = TfidfVectorizer(min_df=5, max_df=0.6,
                                        tokenizer=lambda stream: [token.text for token in tokenize(stream) if
                                                                  token.text not in punctuation])
    print('Results with default tokenizer:\n')
    use_vectorizer(default_vectorizer, train, test)
    print('Results with razdel tokenizer:\n')
    use_vectorizer(custom_vectorizer, train, test)

In [19]:
run_experiment()

Results with default tokenizer:

              precision    recall  f1-score   support

         0.0       0.89      0.90      0.90       969
         1.0       0.80      0.77      0.78       473

    accuracy                           0.86      1442
   macro avg       0.84      0.84      0.84      1442
weighted avg       0.86      0.86      0.86      1442

Results with razdel tokenizer:

              precision    recall  f1-score   support

         0.0       0.89      0.91      0.90       969
         1.0       0.80      0.77      0.78       473

    accuracy                           0.86      1442
   macro avg       0.84      0.84      0.84      1442
weighted avg       0.86      0.86      0.86      1442



Хотя в этом случае результаты показывают незначительное преимущество у токенайзера razdel (по показателю полноты), можно сказать, что разницы практически нет

In [21]:
run_experiment()

Results with default tokenizer:

              precision    recall  f1-score   support

         0.0       0.87      0.90      0.89       946
         1.0       0.80      0.75      0.78       496

    accuracy                           0.85      1442
   macro avg       0.84      0.83      0.83      1442
weighted avg       0.85      0.85      0.85      1442

Results with razdel tokenizer:

              precision    recall  f1-score   support

         0.0       0.87      0.90      0.89       946
         1.0       0.80      0.74      0.77       496

    accuracy                           0.85      1442
   macro avg       0.84      0.82      0.83      1442
weighted avg       0.85      0.85      0.85      1442



В этом эксперименте данные разбились на обучающую и тестовую выборку по-другому, и данные показали преимущество уже у дефолтного токенайзера. Изменения опять же незначительны. Можно сделать вывод, что токенайзер не особо влияет на эффективность модели, гораздо большее значение имеет правильный выбор самой модели. Ну это очевидно. В дефолтном токенайзере используется регулярное выражение "(?u)\b\w\w+\b", и в подавляющем большинстве случаев этого достаточно. Этот токенайзер отработает некорректно, например на словах,в которых есть дефис или апостроф. Даже если считать, что razdel обработает их корректно, такие незначительные случаи вряд ли могут повлиять на итоговый результат. И вообще неизвестно, что лучше: разбивать такие слова на два или оставлять как одно. Всё зависит от решаемой задачи.

## Задание 2

In [28]:
import pandas as pd
from math import log
import re

# инициализируем DataFrame значениями из исходной таблицы, добавляя заголовок первого столбца "Document"
data = pd.DataFrame([['я и ты', 1, 1, 1, 0, 0, 0],
                     ['ты и я', 1, 1, 1, 0, 0, 0],
                     ['я, я, и только я', 3, 0, 1, 1, 0, 0],
                     ['только не я', 1, 0, 0, 1, 1, 0],
                     ['он', 0, 0, 0, 0, 0, 1]],
                    columns=['Document', 'я', 'ты', 'и', 'только', 'не', 'он'])
words = [header for header in data.columns if header != 'Document']  # слова - это все заголовки, кроме "Document"
n = len(data['Document'])  # N Из формулы - общее число документов
dfs = {}  # сюда запишем количества документов, в которых встречается то или иное слово
for word in words:
    data[word] = data[word].astype('float')  # результат будем писать в ту же таблицу, поэтому приводим её к float
    dfs[word] = sum(1 for document in data['Document'] if word in document)  # подсчёт df слова
for ind, row in data.iterrows():  # проходим по документам
    doc_len = len(re.findall(r'\w+', row['Document']))  # определяем число слов в документе
    for word in words:  # проходим по словам в документе
        tf = row[word] / doc_len  # считаем tf из формулы. Число вхождений слова в документ уже есть в таблице.
        # Осталось разделить её на число слов в документе
        data.at[ind, word] = tf * log(n / dfs[word])  # применяем формулу
print(data)


           Document         я       ты         и    только        не        он
0            я и ты  0.074381  0.30543  0.170275  0.000000  0.000000  0.000000
1            ты и я  0.074381  0.30543  0.170275  0.000000  0.000000  0.000000
2  я, я, и только я  0.133886  0.00000  0.102165  0.183258  0.000000  0.000000
3       только не я  0.074381  0.00000  0.000000  0.305430  0.536479  0.000000
4                он  0.000000  0.00000  0.000000  0.000000  0.000000  1.609438


## Задание 3

In [32]:
import pandas as pd
from nltk.corpus import stopwords
from razdel import tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from string import punctuation


def run_model(vectorizer, classifier):
    X_train = vectorizer.fit_transform(train.comment)
    X_test = vectorizer.transform(test.comment)
    y_train = train.toxic.values
    y_test = test.toxic.values
    classifier.fit(X_train, y_train)
    predictions = classifier.predict(X_test)
    print(f'F1 score for classifier "{type(classifier).__name__}" with "{type(vectorizer).__name__}" '
          f'is {f1_score(y_test, predictions)}')
    probabilities = classifier.predict_proba(X_test)
    most_toxic_indices = probabilities[:, 1].argsort()[-10:]
    # probabilities[:, 1] - извлекаем столбец с вероятностями токсичности
    # argsort - сортируем, но получаем индексы, а не вероятности, которые нам не нужны
    # [-10:] - отбираем 10 последних записей (так как сортировка по возрастанию)
    most_toxic = set()
    print("The most toxic comments for this classifier are:\n")
    for i in most_toxic_indices:
        entry = test.iloc[i]  # получаем элемент из тестовой выборки по индексу
        output_message = f'{"TOXIC: " if entry.toxic else "NON-TOXIC"}: {entry.comment.strip()[:100]}...'
        # обрезаем слишком длинные комментарии
        print(output_message)
        most_toxic.add(output_message)
    print()
    return most_toxic


data = pd.read_csv('labeled.csv')
train, test = train_test_split(data, test_size=0.1, random_state=0)
train.reset_index(inplace=True)
test.reset_index(inplace=True)
tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.6, ngram_range=(1, 2), stop_words=stopwords.words('russian'),
                                   tokenizer=lambda stream: [token.text for token in tokenize(stream) if
                                                             token.text not in punctuation])
count_vectorizer = CountVectorizer(min_df=5, max_df=0.6, ngram_range=(1, 2), stop_words=stopwords.words('russian'),
                                   tokenizer=lambda stream: [token.text for token in tokenize(stream) if
                                                             token.text not in punctuation])
naive_bayes = MultinomialNB(fit_prior=False, alpha=1.1)
logistic_regression = LogisticRegression(class_weight='balanced', max_iter=200)
# запускал 4 комбинации,так как по двум непонятно, чем объясняются разные результаты: классификатором или векторайзером
most_toxic_comments = [result for result in [run_model(vectorizer=count_vectorizer, classifier=naive_bayes),
                                             run_model(vectorizer=tfidf_vectorizer, classifier=logistic_regression),
                                             run_model(vectorizer=tfidf_vectorizer, classifier=naive_bayes),
                                             run_model(vectorizer=count_vectorizer, classifier=logistic_regression)]]
print("Comments found by all models:")
for comment in set.intersection(*most_toxic_comments):
    print(comment)


F1 score for classifier "MultinomialNB" with "CountVectorizer" is 0.781316348195329
The most toxic comments for this classifier are:

TOXIC: : Пиздец сука залётная зверушка даже тут насрать успела. Мало того что правил не знает так ещё и права...
NON-TOXIC: Может вас просто собаки бесят?) меня не собаки бесят, а проявление агрессии.. Давить собак - проявля...
TOXIC: : Не зря, вас, хохлов, свиньями кличут. Вы и есть грязные животные, не способные к любви, привязанност...
TOXIC: : Ты либо не знаешь правил русского языка либо шизофреник. Нет - предикатив, отрицающий факт существов...
TOXIC: : В России два пола. Гомосеком быть стыдно и опасно. Негры - это негры, а не афроктототам. Женщины име...
TOXIC: : ДА КАКОГО ЕБАНОГО ХУЯ МНЕ ТЕПЕРЬ ЮТУБ РЕКОМЕНДУЕТ ЕБУЧЕГО ШЕВЦОВА НАХУЙ СУКА БЛЯДЬ? Я КЛЯНУСЬ ЖОПОЙ ...
NON-TOXIC: Раз уж вакханалия продолжается, давайте в этом треде продолжать собирать ворох банов. Копирую из про...
NON-TOXIC: та ну, хуйня это все про джентельменство . меня в свое время

В целом, можно заметить, что результаты на разных моделях отличаются. "Единодушно" самым токсичным было определено только два коментария. Заметим, модели на основе TF-IDF Vectorizer безошибочно отнесли комментарии в число самых токсичных (как в байесовском классификаторе, так и в логистической регрессии). А вот модели на CountVectorizer отнесли в число самых токсичных по 2-3 обычных комментария. Можно заметить, что эти ошибочные комментарии также содержат грубую лексику, поэтому ошибка объяснима. Однако, отнесение этих примеров к "самым токсичным" наводит на мысль, что TF-IDF Vectorizer в этом плане гораздо эффективнее (если он решил, что текст токсичный, значит, так оно и есть).

## Задание 4

In [36]:
import pandas as pd
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier


def run_model(vectorizer, classifier):
    X_train = vectorizer.fit_transform(train.comment)
    y_train = train.toxic.values
    classifier.fit(X_train, y_train)
    classifier_name = type(classifier).__name__
    inverse_vocabulary = {value: key for key, value in vectorizer.vocabulary_.items()}  # для удобства поиска по индексу
    # поменяем местами индексы и слова
    print(f'Most important words for {classifier_name}:')
    most_important_features = []
    if classifier_name == 'LogisticRegression':
        most_important_features = classifier.coef_[0].argsort()[-1:-6:-1]
        # classifier.coef_[0] - извлекаем таблицу важности признаков
        # argsort - сортируем и сразу получаем индексы
        # [-1:-6:-1] - берем последние 5 элементов (так как сортировка по возрастанию) в перевернутом виде (чтобы самое
        # важное слово было первым)
    elif classifier_name == 'MultinomialNB':
        most_important_features = classifier.feature_log_prob_[1].argsort()[-1:-6:-1]
    elif classifier_name in ['RandomForestClassifier', 'DecisionTreeClassifier']:
        most_important_features = classifier.feature_importances_.argsort()[-1:-6:-1]
    print([inverse_vocabulary[feature_index] for feature_index in most_important_features], "\n")


data = pd.read_csv('labeled.csv')
train, test = train_test_split(data, test_size=0.1, random_state=0)
train.reset_index(inplace=True)
test.reset_index(inplace=True)
tfidf_vectorizer = TfidfVectorizer(min_df=40, max_df=0.025, stop_words=stopwords.words('russian'), ngram_range=(1, 3))
models = [DecisionTreeClassifier(class_weight='balanced'),
          RandomForestClassifier(),
          LogisticRegression(class_weight='balanced'),
          MultinomialNB(fit_prior=False)]
for model in models:
    run_model(vectorizer=tfidf_vectorizer, classifier=model)


Most important words for DecisionTreeClassifier:
['тебе', 'хохлы', 'хохлов', 'нахуй', 'например'] 

Most important words for RandomForestClassifier:
['хохлы', 'тебе', 'хохлов', 'нахуй', 'блядь'] 

Most important words for LogisticRegression:
['хохлов', 'хохлы', 'дебил', 'русских', 'сука'] 

Most important words for MultinomialNB:
['тебе', 'хохлы', 'хохлов', 'нахуй', 'хуй'] 

